In [11]:
# 5- Julia replicar la función OLS presentada en clase (ultima función del laboratorio de R y python)

# 20180783	Romina Garibay
# 20183566	Marissa Vergara
# 20163164	Lisbeth Morales

In [2]:
# import Pkg
# Pkg.add("LinearAlgebra")
# Pkg.add("Random")
# Pkg.add("Distributions")
# Pkg.add("Statistics")
# Pkg.add("DataFrames")

In [3]:
using Random  
using Statistics 
using Distributions, LinearAlgebra
using DataFrames

In [4]:
using Random
using Distributions
Random.seed!(756)

x1 = rand(500)  # uniform distribution
x2 = rand(500) # uniform distribution
x3 = rand(500) # uniform distribution
x4 = rand(500) # uniform distribution

e = randn(500) # normal dsitribution mean = 0 y sd = 1

# DGP

Y = ones(500) + 0.8.*x1 + 1.2.*x2 + 0.5.*x3 + 1.5.*x4 + e

500-element Vector{Float64}:
 3.651760111637669
 2.179257931872106
 2.692229249483935
 3.3469728701923085
 3.5588447837576
 2.276211694590222
 4.829946629006251
 4.093224741375721
 3.619062546112381
 3.626039311588756
 3.6295199167710184
 4.699162411600479
 2.97187185098835
 ⋮
 2.143054993101037
 5.156448840518553
 5.552718346568406
 4.509710634153028
 2.8803764549617084
 1.4650621858976582
 5.2641481243982975
 2.856488034023155
 2.118228555438558
 2.7524244417837345
 2.530006799870474
 2.7102224930461163

In [5]:
X = hcat(ones(500),x1,x2,x3,x4)

500×5 Matrix{Float64}:
 1.0  0.838562    0.818703  0.873998   0.307697
 1.0  0.35974     0.857487  0.321217   0.402396
 1.0  0.866428    0.376773  0.116841   0.01289
 1.0  0.620758    0.110184  0.773056   0.473527
 1.0  0.790826    0.380551  0.3113     0.576488
 1.0  0.814543    0.165241  0.826491   0.912464
 1.0  0.294621    0.987889  0.372204   0.312046
 1.0  0.991318    0.493014  0.852898   0.471054
 1.0  0.660835    0.318123  0.639298   0.921224
 1.0  0.776585    0.137327  0.939525   0.927768
 1.0  0.735151    0.57674   0.892356   0.729053
 1.0  0.265968    0.882513  0.977335   0.78393
 1.0  0.805829    0.638077  0.0594682  0.51971
 ⋮                                     
 1.0  0.0733076   0.216207  0.810691   0.545487
 1.0  0.752575    0.556695  0.833254   0.364712
 1.0  0.991432    0.772759  0.717453   0.780103
 1.0  0.704278    0.528964  0.0236845  0.876592
 1.0  0.426315    0.628536  0.906187   0.265348
 1.0  0.947555    0.561127  0.392688   0.658197
 1.0  0.86817     0.969904  

In [6]:
function ols(M,Y, standar = true, Pvalue::Bool = true , instrumento = nothing, index = nothing)

    if standar && Pvalue && isnothing(instrumento) && isnothing(index)     #2. si se cumple todo, mostrar
        
        
        beta = inv(transpose(X) * X) * (transpose(X) * Y)                        #2.a. estimar beta
        
        y_est =  X * beta                                                        #2.b. Y estimado = vector 
        
        n  = size(X,1)                                                           #2.c. numero de observaciones
        
        k  = size(X,2)  - 1                                                      #2.d. numeros de x - unos (grados de libertad)
        
        nk = n - k                                                               #2.e. grados de libertad
        
        ee= Y - y_est
        
        sigma =  sum(ee.^2 ) / nk                                                #2.f. SCR/ (n-k) = e'e /(n-k) = s^2 =sigma^2
        
        Var   = sigma*inv(transpose(X) * X)                                      #2.g. Var_Cov(betas) = s^2 * (X'X)-1
        sd    = Var[diagind(Var)].*(1/2)                                         #2.h. desv(betas)=var(betas)^1/2 , solo diag
         
        t_est = abs.(beta./sd)                                                   #2.i. t_est = |beta-0|/sd ==> para cada beta 
        dist= TDist(nk)
        
        pvalue= (1 .- cdf(dist, t_est) ).*2
        df    = DataFrame(OLS= beta , standar_error= sd, P_value=pvalue )    

    
    elseif !isnothing(instrumento) && !isnothing(index)            #2. Si el instrumento no es None   &
                                                                                      #   el índice no es None
        
        beta = inv(transpose(X) * X) * (transpose(X) * Y)                  #2.a. estimar beta sin corregir endogeneidad
        
        index = index  - 1                                                 #2.b. xk con endogeneidad está en fila index
                                                                           #     que sería index - 1
        Z = X
        Z[:,index] = z                                                     #2.c. reemplazar toda la columna de xk endogena con z
    
        
        
        beta_x = inv(transpose(Z) * Z) * (Transpose(Z) * X[:,index] )            #2.d. estimar beta de xk con Z (= demas Xs + instrumento)          
        x_est  = Z * beta_x                                                #2.e. estimar xk_estimado
        
        
        X[:,index] = x_est                                                 #2.f. reemplazar la variable xk por su estimado xk_estimado 
        beta_iv = inv(transpose(X) * X) * (transpose(X) * Y )                    #2.a. estimar beta corrigiendo endogeneidad
        
        
        df = DataFrame(OLS= beta , OLS_IV= beta_iv)  

    return df
    end
end

ols (generic function with 5 methods)

In [7]:
ols(X,Y)

,OLS,standar_error,P_value
,Float64,Float64,Float64
1,0.63834,0.0135565,0.0
2,0.846653,0.0131005,0.0
3,1.55048,0.0123729,0.0
4,0.743114,0.0131288,0.0
5,1.78192,0.0122803,0.0
